# Trabajando con texto

Seguramente habrás escuchado hablar de algo llamado ChatGPT. Pues aquí tiene sus principios: en el análisis de texto.

El análisis de texto es un tema que merece probablemente su propio curso, pero podemos ir poniendo las bases empleando scikit-learn.

Lo primero, es que hay que recordar que las palabras escritas no son consumibles directamente por los modelos de aprendizaje automático, sino que necesitamos convertirlas en una representación numérica que podamos procesar. 

## Bag of words – CountVectorizer

Una de las formas más comunes de hacer esto es mediante el uso de la técnica de bag of words (BoW) o bolsa de palabras, que convierte un texto en una matriz de frecuencia de palabras.

Para llevar a cabo este scikit-learn nos ofrece una utilidad que realiza por nosotros lo siguiente:

 - Parte el texto en tokens, generalmente palabras completas,

 - Cuenta las ocurrencias de cada uno de estos tokens,

 - Asigna valores dentro de un vector de acuerdo a el número de ocurrencias de cada token en nuestros datos de entrada.

Esto lo hace a través del vectorizador conocido como <code>CountVectorizer</code>, para verlo en acción, primero hay que generar un conjunto de datos. Por cierto, a un conjunto de datos se le suele llamar corpus, y a cada uno de sus elementos individuales se le conoce como documento. 

Entonces vamos a generar un corpus con tres documentos:

In [ ]:
corpus = [
    "Scikit-learn nos ayuda a trabajar con texto",
    "Parte el texto en tokens, generalmente palabras completas",
    "Cuenta las ocurrencias de cada uno de estos tokens"
]

Importamos el vectorizador – nota que estamos importando de <code>sklearn.feature_extraction.text</code>:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Creamos un objeto con valores por default – y lo entrenamos con nuestro corpus:

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(corpus)

Si llamamos al método <code>transform</code> pasándole nuestro corpus, el resultado es el esperado: una matriz dispersa puesto que esa es la mejor representación de nuestros datos. Podemos convertirla a un arreglo de NumPy con su método <code>todense</code>.

In [ ]:
transformed_corpus = count_vectorizer.transform(corpus)
transformed_corpus.todense()

Sí, es una matriz con un montón de ceros. Si quieres ver a qué palabra corresponde cada columna, puedes acceder a la propiedad calculada <code>vocabulary_</code>, así podrás ver que la palabra “ayuda” corresponde a la columna cero y que la palabra “uno” corresponde a la columna número 20.

## Transformación inversa

El <code>CountVectorizer</code> nos ofrece el método <code>inverse_transform</code> para que a partir de una matriz de vectores, puedas recuperar los tokens que usaste a la entrada. Ten cuidado porque a pesar de que es una transformación inversa, esta no es tan fidedigna, puesto que una de las desventajas de esta familia de vectorizadores es que el orden de las palabras se pierde. 

Lo podemos probar llamando al método con la matriz que acabamos de conseguir:

In [ ]:
count_vectorizer.inverse_transform(transformed_corpus)

## Parámetros extra

El <code>CountVectorizer</code> es una de las primeras clases que tiene una gran cantidad de parámetros para configurar su comportamiento. Entre las más comunes que he viso son usadas están:

 - <code>binary</code> que por defecto tiene el valor de <code>False</code>, cuando este valor es verdadero, el <code>CountVectorizer</code> se comporta más bien como un one-hot encoder, nuestra matriz resultante consta de unos y ceros.

 - <code>max_features</code>, este puede ser un número que nos indica la cantidad máxima de columnas que queremos dentro de nuestra matriz. En el ejemplo anterior teníamos como resultado una matriz de 21 columnas, pero si hubiésemos establecido <code>max_features</code> con un valor de 10, tendríamos como resultado una matriz de 10 columnas, en donde esas diez columnas contendrían los 10 tokens más frecuentes.

 - <code>max_df</code> y <code>min_df</code>, estos parámetros nos permiten eliminar palabras que están sobrerepresentadas y subrepresentadas en nuestro corpus. Estos valores pueden ser flotantes, pueden ir entre 0 y 1 si queremos usarlo como proporción, o podemos usarlo como entero si queremos contar las ocurrencias netamente.

Por ejemplo, si creamos un vectorizador con los siguientes argumentos:

In [ ]:
modified_count_vectorizer = CountVectorizer(
    binary = True,
    max_features = 6,
    min_df = 1
)

Podrás ver que como resultado de transformar nuestro corpus obtenemos una matriz de 6 columnas, rellena con solamente unos y ceros – y que el vocabulario está formado únicamente de los seis tokens con mayor frecuencia:

In [ ]:
new_result = modified_count_vectorizer.fit_transform(corpus)
print(modified_count_vectorizer.vocabulary_)
new_result.todense()

## Cambiando el tokenizer

Hay ocasiones en las que queremos tener más control sobre la forma en la que los documentos de nuestro corpus deben ser fragmentados en tokens. Por ejemplo si estamos lidiando con otro idioma, o estamos trabajando con texto que contiene emojis.

In [ ]:
import re

# Un tokenizador que mantiene solo los emojis
def emoji_tokenizer(text):
    emojis = re.findall(r'[\U0001F000-\U0001F6FF]', text)
    return emojis

print(emoji_tokenizer("I 💚 🍕"))

Usamos el tokenziador pasándolo a <code>CountVectorizer</code> en el argumento <code>tokenizer</code>:

In [ ]:
emoji_corpus = [
    "I 💚 🍕",
    "This 🍕 was 👎",
    "I like either 🍕 or 🍔, but not 🌭",
]

emoji_vectorizer = CountVectorizer(tokenizer=emoji_tokenizer)
X = emoji_vectorizer.fit_transform(emoji_corpus)

# Print the feature names and the count matrix
print(emoji_vectorizer.vocabulary_)
print(X.toarray())

## Ponderación Tf-idf – TfidfVectorizer

En un corpus de texto grande, algunas palabras estarán muy sobrerrepresentadas (como "el", "un", "es") y, por lo tanto, tienen poca información significativa sobre el contenido real del documento – si estas palabras existen en todos los documentos, no son tan útiles.

Si nosotros tomamos los resultados de un <code>CountVectorizer</code> en esos casos, corremos el riesgo de pasarle información innecesaria a nuestro modelo, ocultando así palabras menos frecuentes, más raras y más interesantes. Para abordar este problema, podemos utilizar técnicas de extracción de características de texto que ponderan las palabras en función de su importancia relativa en el corpus.

Una técnica común utilizada para la extracción de características de texto es la frecuencia de término – frecuencia inversa de documento o (TF-IDF), que mide la importancia relativa de una palabra en un documento en función de la frecuencia de esa palabra en el corpus en su conjunto.

Scikit-learn nos ofrece una clase llamada <code>TfidfVectorizer</code> que tiene el mismo comportamiento externo que el <code>CountVectorizer</code>: recibe un conjunto de textos y nos da una matriz correspondiente. Además de que tiene casi los mismos argumentos. 

 > 📚 De tarea se te queda probar los ejemplos que vimos pero utilizando el <code>TfidfVectorizer</code>, dime en los comentarios ¿qué notas de diferencia?

## Feature hashing

Otra de las formas de convertir texto a representación numérica es a través del uso del <i>hashing trick</i>, ¿recuerdas que lo vimos hace poco? – para este caso, scikit-learn nos tiene otra clase: <code>HashingVectorizer</code>, que comparte muchas características con los dos vectorizadores anteriormente vistos. Obviamente, tiene las mismas limitantes que ya conocemos, sin embargo puede ser buena alternativa en algunos escenario – ah, recuerda, utilizando <i>feature hashing</i> es imposible regresar a los valores originales.

 > 📚 Otra tarea más: prueba los ejemplos que vimos pero utilizando el <code>HashingVectorizer</code>, dime en los comentarios ¿qué notas de diferencia?

## Conclusión

Ya vimos múltiples maneras de transformar nuestro texto en números, <code>CountVectorizer</code> en modo cuenta o en modo binario, <code>TfidfVectorizer</code> y <code>HashingVectorizer</code>. El método que debas usar dependerá de tu caso de uso, pero puedes seguir estas reglas generales:

 - <code><b>CountVectorizer</b></code> es útil para crear una matriz de recuento de palabras cuando es importante el número absoluto de ocurrencias de cada palabra en el texto.

 - <code><b>TfidfVectorizer</b></code> es útil cuando se pondera la importancia de cada palabra en el texto en función de la frecuencia con que aparece en el corpus.

 - <code><b>HashingVectorizer</b></code> es útil para trabajar con conjuntos de datos muy grandes que no caben en memoria y para reducir la dimensionalidad del espacio de características.

Espero ahora te quede más claro cuál es el primer paso antes de tratar de alimentar texto en tus modelos de machine learning, recuerda que debes practicar y en los recursos encontrarás ejemplos prácticos en los que se usan algunos vectorizadores.